# Lab Assignment #4 – Using ADT Stacks, Queues, and Lists

## Exercise 1

**If your first name starts with a letter from A-J inclusively:**

Suppose we want to extend the **PositionalList** ADT with a method,
***indexOf(p)***, that returns the current index of the element stored
at position p. Write this method using only other methods of the
**PositionalList** interface (not details of our LinkedPositionalList
implementation).

Write the necessary code to test the method. **Hint:** Count the steps
while traversing the list until encountering position p.

**If your first name starts with a letter from K-Z inclusively:**

Suppose we want to extend the PositionalList abstract data type with a
method, ***findPosition(e)***, that returns the first position
containing an element equal to e (or null if no such position exists).

Implement this method using only existing methods of the PositionalList
interface (not details of our LinkedPositionalList implementation).
Write the necessary code to test the method.

**Hint:** use the *equals* method to test equality.

In [31]:
# Copyright 2013, Michael H. Goldwasser
#
# Developed for use with the book:
#
#    Data Structures and Algorithms in Python
#    Michael T. Goodrich, Roberto Tamassia, and Michael H. Goldwasser
#    John Wiley & Sons, 2013
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

from exercise1.doubly_linked_base import _DoublyLinkedBase


class PositionalList(_DoublyLinkedBase):
    """A sequential container of elements allowing positional access."""

    # -------------------------- nested Position class --------------------------
    class Position:
        """An abstraction representing the location of a single element.

        Note that two position instaces may represent the same inherent
        location in the list.  Therefore, users should always rely on
        syntax 'p == q' rather than 'p is q' when testing equivalence of
        positions.
        """

        def __init__(self, container, node):
            """Constructor should not be invoked by user."""
            self._container = container
            self._node = node

        def element(self):
            """Return the element stored at this Position."""
            return self._node._element

        def __eq__(self, other):
            """Return True if other is a Position representing the same location."""
            return type(other) is type(self) and other._node is self._node

        def __ne__(self, other):
            """Return True if other does not represent the same location."""
            return not self == other  # opposite of __eq__

    # ------------------------------- utility methods -------------------------------
    def _validate(self, p):
        """Return position's node, or raise appropriate error if invalid."""
        if not isinstance(p, self.Position):
            raise TypeError("p must be proper Position type")
        if p._container is not self:
            raise ValueError("p does not belong to this container")
        if p._node._next is None:  # convention for deprecated nodes
            raise ValueError("p is no longer valid")
        return p._node

    def _make_position(self, node):
        """Return Position instance for given node (or None if sentinel)."""
        if node is self._header or node is self._trailer:
            return None  # boundary violation
        return self.Position(self, node)  # legitimate position

    # ------------------------------- accessors -------------------------------
    def first(self):
        """Return the first Position in the list (or None if list is empty)."""
        return self._make_position(self._header._next)

    def last(self):
        """Return the last Position in the list (or None if list is empty)."""
        return self._make_position(self._trailer._prev)

    def before(self, p):
        """Return the Position just before Position p (or None if p is first)."""
        node = self._validate(p)
        return self._make_position(node._prev)

    def after(self, p):
        """Return the Position just after Position p (or None if p is last)."""
        node = self._validate(p)
        return self._make_position(node._next)

    def __iter__(self):
        """Generate a forward iteration of the elements of the list."""
        cursor = self.first()
        while cursor is not None:
            yield cursor.element()
            cursor = self.after(cursor)

    # ------------------------------- mutators -------------------------------
    # override inherited version to return Position, rather than Node
    def _insert_between(self, e, predecessor, successor):
        """Add element between existing nodes and return new Position."""
        node = super()._insert_between(e, predecessor, successor)
        return self._make_position(node)

    def add_first(self, e):
        """Insert element e at the front of the list and return new Position."""
        return self._insert_between(e, self._header, self._header._next)

    def add_last(self, e):
        """Insert element e at the back of the list and return new Position."""
        return self._insert_between(e, self._trailer._prev, self._trailer)

    def add_before(self, p, e):
        """Insert element e into list before Position p and return new Position."""
        original = self._validate(p)
        return self._insert_between(e, original._prev, original)

    def add_after(self, p, e):
        """Insert element e into list after Position p and return new Position."""
        original = self._validate(p)
        return self._insert_between(e, original, original._next)

    def delete(self, p):
        """Remove and return the element at Position p."""
        original = self._validate(p)
        return self._delete_node(original)  # inherited method returns element

    def replace(self, p, e):
        """Replace the element at Position p with e.

        Return the element formerly at Position p.
        """
        original = self._validate(p)
        old_value = original._element  # temporarily store old element
        original._element = e  # replace with new element
        return old_value  # return the old element value

    def index_of(self, p):
        """Return the index of the element stored at position p."""
        index = 0
        current_position = self.first()
        while current_position is not None:
            if current_position == p:
                return index
            index += 1
            current_position = self.after(current_position)
        raise ValueError("Position not found")

    def find_position(self, e):
        """Return the first element with value e or None if not found."""
        cursor = self.first()
        while cursor:
            if cursor.element() == e:
                return cursor
            cursor = self.after(cursor)
        return None


def main():
    pl = PositionalList()
    pos1 = pl.add_last(10)
    pos2 = pl.add_last(20)
    pos1 = pl.add_last(10)
    pos1 = pl.add_last(10)
    pos1 = pl.add_last(10)
    pos3 = pl.add_last(30)
    pos4 = pl.add_last(40)
    
    print(pl.find_position(10))
    print(pl.first())
    print(pl.index_of(pl.find_position(10)))

    print(pl.find_position(30))
    print(pl.first())
    print(pl.index_of(pl.find_position(30)))


if __name__ == "__main__":
    main()


0
5


(5 marks)

## Exercise 2

**If your first name starts with a letter from A-J inclusively:**

Implement a method with signature *transfer(S, T)* that transfers all
elements from stack S onto stack T, so that the element that starts at
the top of S is the first to be inserted onto T, and the element at the
bottom of S ends up at the top of T. Write the necessary code to test
the method.

**If your first name starts with a letter from K-Z inclusively:**

Write a recursive method for removing all the elements from a stack S.
Write the necessary code to test the method.

**Hint:** First check if the stack is already empty.

In [32]:
class Stack:
    def __init__(self):
        self._elements = []

    def size(self):
        return len(self._elements)

    def is_empty(self):
        return len(self._elements) == 0

    def push(self, e):
        self._elements.append(e)

    def top(self):
        if self.is_empty():
            raise IndexError("Top from an empty stack")
        return self._elements[-1]

    def pop(self):
        if self.is_empty():
            raise IndexError("Pop from an empty stack")
        return self._elements.pop()

    def transfer(self, T):
        while not self.is_empty():
            element = self.pop()
            T.push(element)

    def remove_all(self):
        """
        Recursively removes all elements from the given stack.
        """
        if not self.is_empty():
            self.pop()
            self.remove_all()


def remove_all_elements(stack):
    """
    Recursively removes all elements from the given stack.
    """
    # Base case: if the stack is empty, return (stop recursion)
    if stack.is_empty():
        return

    # Recursive case: pop an element and call remove_all_elements again
    stack.pop()
    remove_all_elements(stack)


def main():
    stack1 = Stack()
    stack2 = Stack()

    # Push elements into stack1
    stack1.push(1)
    stack1.push(2)
    stack1.push(3)
    print(f"Stack 1 before transfer: {stack1._elements}")
    print(f"Stack 2 before transfer: {stack2._elements}")

    # Transfer elements from stack1 to stack2
    stack1.transfer(stack2)
    print(f"Stack 1 after transfer: {stack1._elements}")
    print(f"Stack 2 after transfer: {stack2._elements}")

    # Remove all elements from stack2
    stack2.remove_all()
    print(f"Stack 2 after removing all elements: {stack2._elements}")


if __name__ == "__main__":
    main()

Stack 1 before transfer: [1, 2, 3]
Stack 2 before transfer: []
Stack 1 after transfer: []
Stack 2 after transfer: [3, 2, 1]
Stack 2 after removing all elements: []


(2 marks)

## Exercise 3

Implement a method with signature ***concatenate(LinkedQueue\<E\> Q2)***
for the **LinkedQueue\<E\>** class that takes all elements of Q2 and
appends them to the end of the original queue. The operation should run
in **O(1)** time and should result in Q2 being an empty queue. Write the
necessary code to test the method. **Hint:** You may just modify the
SinglyLinkedList class to add necessary support.

(3 marks)